In [63]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split

In [71]:
df = pd.read_csv('../data/raw/nytaxi2022.csv', usecols=[
     "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", "trip_distance", 
     "RatecodeID", "PULocationID", "DOLocationID", "payment_type", 
     "extra", "total_amount"], nrows = 4000000)
print(df.shape)
# df.head()

(4000000, 10)


In [72]:
#df = df.drop_duplicates()
df = df.dropna()
df = df[df["passenger_count"] > 0]
df = df[df["trip_distance"] > 0]
df = df[df["total_amount"] > 0]
df = df[df['PULocationID'].between(1, 265)] #这两行好像没丢掉数据
df = df[df['DOLocationID'].between(1, 265)]


df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"], format = "%m/%d/%Y %H:%M:%S %p")
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"], format = "%m/%d/%Y %H:%M:%S %p")
trip_dur = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
df = df[trip_dur > 0]
 

df.shape

(3718676, 10)

In [22]:
#df.head()

In [73]:
#时间
df["pickup_ts"] = df["tpep_pickup_datetime"].astype('int64') // 10**9
df["dropoff_ts"] = df["tpep_dropoff_datetime"].astype('int64') // 10**9
df = df.drop(columns=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

# 类别onehot
# cat_cols = ['RatecodeID','payment_type','PULocationID','DOLocationID']
cat_cols = ['RatecodeID','payment_type']
# df = pd.get_dummies(df, columns=cat_cols, dummy_na=False)
for col in cat_cols:
    dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False)
    df = pd.concat([df, dummies], axis=1)
    df.drop(columns=[col], inplace=True)

num_cols = ['passenger_count','trip_distance','extra','pickup_ts','dropoff_ts', 'PULocationID','DOLocationID']
for col in num_cols:
    col_min = df[col].min()
    col_max = df[col].max()
    df[col] = (df[col] - col_min) / (col_max - col_min)

df.shape
# df.head()

(3718676, 20)

In [74]:
df.head()

,passenger_count,trip_distance,PULocationID,DOLocationID,extra,total_amount,pickup_ts,dropoff_ts,RatecodeID_1.0,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_6.0,RatecodeID_99.0,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5
0,0.125,0.005822,0.534091,0.890152,0.122449,21.95,0.970826,0.970828,True,False,False,False,False,False,False,True,False,False,False,False
1,0.000,0.003210,0.890152,0.155303,0.020408,13.30,0.970826,0.970827,True,False,False,False,False,False,False,True,False,False,False,False
3,0.000,0.001659,0.428030,0.253788,0.020408,11.80,0.970825,0.970826,True,False,False,False,False,False,False,False,True,False,False,False
6,0.000,0.007773,0.878788,0.325758,0.020408,26.00,0.970824,0.970826,True,False,False,False,False,False,False,True,False,False,False,False
7,0.000,0.003088,0.897727,0.571970,0.020408,12.80,0.970823,0.970824,True,False,False,False,False,False,False,False,True,False,False,False


In [75]:
cols = [c for c in df.columns if c != "total_amount"] + ["total_amount"]
df = df[cols]

In [76]:
df.head()

,passenger_count,trip_distance,PULocationID,DOLocationID,extra,pickup_ts,dropoff_ts,RatecodeID_1.0,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_6.0,RatecodeID_99.0,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,total_amount
0,0.125,0.005822,0.534091,0.890152,0.122449,0.970826,0.970828,True,False,False,False,False,False,False,True,False,False,False,False,21.95
1,0.000,0.003210,0.890152,0.155303,0.020408,0.970826,0.970827,True,False,False,False,False,False,False,True,False,False,False,False,13.30
3,0.000,0.001659,0.428030,0.253788,0.020408,0.970825,0.970826,True,False,False,False,False,False,False,False,True,False,False,False,11.80
6,0.000,0.007773,0.878788,0.325758,0.020408,0.970824,0.970826,True,False,False,False,False,False,False,True,False,False,False,False,26.00
7,0.000,0.003088,0.897727,0.571970,0.020408,0.970823,0.970824,True,False,False,False,False,False,False,False,True,False,False,False,12.80


In [77]:
import os

os.makedirs("../data/processed", exist_ok=True)

# 保存为 parquet
df.to_parquet("../data/processed/data4000000.parquet", index=False)
print("Saved to ../data/processed/data4000000.parquet")



Saved to ../data/processed/data4000000.parquet


In [78]:
import pyarrow.parquet as pq

# 读 parquet 文件的元信息
parquet_file = pq.ParquetFile("../data/processed/data4000000.parquet")

print("Number of row groups:", parquet_file.num_row_groups)

# 查看每个 row group 的大小和行数
for i in range(parquet_file.num_row_groups):
    rg_meta = parquet_file.metadata.row_group(i)
    print(f"Row Group {i}: {rg_meta.num_rows} rows, {rg_meta.total_byte_size/1024/1024:.2f} MB")


Number of row groups: 4
Row Group 0: 1048576 rows, 23.08 MB
Row Group 1: 1048576 rows, 22.99 MB
Row Group 2: 1048576 rows, 22.82 MB
Row Group 3: 572948 rows, 11.29 MB


In [79]:
parquet_file.read_row_group(1).to_pandas().head()

,passenger_count,trip_distance,PULocationID,DOLocationID,extra,pickup_ts,dropoff_ts,RatecodeID_1.0,RatecodeID_2.0,RatecodeID_3.0,RatecodeID_4.0,RatecodeID_5.0,RatecodeID_6.0,RatecodeID_99.0,payment_type_1,payment_type_2,payment_type_3,payment_type_4,payment_type_5,total_amount
0,0.000,0.001214,0.617424,0.253788,0.122449,0.973671,0.973671,True,False,False,False,False,False,False,True,False,False,False,False,10.30
1,0.000,0.005361,0.337121,0.606061,0.122449,0.973673,0.973675,True,False,False,False,False,False,False,True,False,False,False,False,23.75
2,0.125,0.001214,0.890152,0.901515,0.122449,0.973668,0.973668,True,False,False,False,False,False,False,True,False,False,False,False,9.55
3,0.125,0.009816,0.534091,0.242424,0.122449,0.973672,0.973676,True,False,False,False,False,False,False,True,False,False,False,False,33.35
4,0.000,0.001413,0.939394,0.871212,0.020408,0.973669,0.973670,True,False,False,False,False,False,False,True,False,False,False,False,10.04


In [80]:
df = pd.read_parquet("../data/processed/data4000000.parquet")  
print(df["total_amount"].min())  
print(df["total_amount"].max())  
print(df["total_amount"].mean())  
print(df["total_amount"].std())  

0.13
401095.62
18.964863795608977
208.45984281933949


In [ ]:
# # 时间转 Unix 秒
# df["pickup_ts"] = df["tpep_pickup_datetime"].astype('int64') // 10**9
# df["dropoff_ts"] = df["tpep_dropoff_datetime"].astype('int64') // 10**9
# df = df.drop(columns=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

# # 类别onehot
# cat_cols = ['RatecodeID','payment_type','PULocationID','DOLocationID']
# df = pd.get_dummies(df, columns=cat_cols, dummy_na=False)

# # 数值归一化
# num_cols = ['passenger_count','trip_distance','extra','pickup_ts','dropoff_ts']
# scaler = MinMaxScaler()
# df[num_cols] = scaler.fit_transform(df[num_cols])

# df.head()

: 